In [1]:
from time import time
import numpy as np 
import pandas as pd 

import re
import nltk
from nltk.corpus import stopwords
from gensim.models import word2vec

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("labeled_rutoxic.csv", delimiter=',', header=0, names=['sentence', 'label'])

print('В наборе предложений: \n',df.shape[0])

В наборе предложений: 
 14412


In [3]:
print('toxic:', df[df['label'] > 0]['label'].count())
print('not toxic:', df[df['label'] < 1]['label'].count())

toxic: 4826
not toxic: 9586


## Разбиение на тестовые и обучающие

In [4]:
X = df.iloc[:,0]# 
y = df.iloc[:,1]# 

train , test , y_train, y_test = train_test_split(X, y, test_size=0.3) # и отдаем 30% на тест, остальное на обучен

## Обучение модели Word2Vec

In [5]:
# Метод преобразования текста в слова
def text_to_words(raw_text, remove_stopwords=False):
   
    letters_only = re.sub("[^0-9а-яА-Я]", " ", raw_text)
    words = letters_only.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("russian"))
        meaningful_words = [w for w in words if not w in stops]
        words = meaningful_words
    return words 

sentences_train = train.apply(text_to_words, remove_stopwords=False)
sentences_test = test.apply(text_to_words, remove_stopwords=False)
print(sentences_train[:1])

10101    [в, магазине, 15, касс, и, все, работают]
Name: sentence, dtype: object


In [6]:

num_features = 300                
min_word_count = 40                     
num_workers = 4      
context = 20                                                                                        
downsampling = 1e-3  
model = word2vec.Word2Vec(sentences_train, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)

t = time()
model.init_sims(replace=True) 
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))


Time to build vocab: 0.0 mins


C:\Users\gross\AppData\Local\Temp\ipykernel_13480\2841341794.py:9: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [7]:
# получение векторного представления
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    index2word_set = set(model.wv.index_to_key)
   
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec, model.wv[word])

    if nwords == 0:
        nwords = 1
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# получение среднего векторного простнраства для предложения
def getAvgFeatureVecs(reviews, model, num_features):
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    counter = 0
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
    return reviewFeatureVecs

f_matrix_train = getAvgFeatureVecs(sentences_train, model, num_features)
f_matrix_test = getAvgFeatureVecs(sentences_test, model, num_features)
print(f_matrix_train[0,:])

[ 1.22797564e-02  1.00694202e-01 -1.55817959e-02  5.09373434e-02
  7.47452443e-03 -1.30126879e-01  4.61097322e-02  1.69293433e-01
  3.32843629e-03 -7.33366981e-02 -4.55685100e-03 -6.90163374e-02
  3.06101441e-02 -1.48919420e-02 -5.88536859e-02 -1.06256522e-01
  4.09237072e-02  1.79147869e-02  6.70788661e-02 -1.57195441e-02
 -3.78481969e-02 -7.59452432e-02  2.90623959e-02  3.96989062e-02
  3.81834693e-02 -5.57479337e-02 -1.28999144e-01 -3.11287753e-02
 -6.17939159e-02 -4.79364395e-02  1.96185317e-02 -9.17814597e-02
  7.49859959e-02 -1.36337858e-02 -3.14146914e-02  3.35382186e-02
  9.78233479e-03 -9.06202048e-02 -2.21713148e-02  1.15315020e-02
 -4.06706408e-02  2.87830196e-02 -5.19797280e-02 -3.25058363e-02
  1.60648003e-02  8.65984708e-02 -1.59603097e-02  1.73982885e-02
  1.93748511e-02  4.76965047e-02  4.74498458e-02 -1.25044063e-02
 -4.50505055e-02  6.30372379e-04  5.94289089e-03  7.46942386e-02
  3.01601104e-02 -3.79205570e-02 -1.06896870e-02 -2.11461000e-02
 -5.50598279e-02 -1.67070

## Создание обучение  MLPClassifier

In [8]:
model = []

m = MLPClassifier(solver='adam', hidden_layer_sizes=(300,40,30), random_state=1)
model.append(m)

print(model)

[MLPClassifier(hidden_layer_sizes=(300, 40, 30), random_state=1)]


In [9]:
batch_size = 200
total_rows = f_matrix_train.shape[0]
duration = 0
start_train = time()
pos = 0
classes = [0.0, 1.0]
while duration < 10 and pos < total_rows:
    if pos+batch_size > total_rows:
        batch_size = total_rows-pos
    X_p = f_matrix_train[pos:pos+batch_size]
    y_p = y_train.values[pos:pos+batch_size]
    model[0].partial_fit(X_p, y_p, classes)
    pos = pos + batch_size
    duration = time() - start_train
    if pos == total_rows:
        pos = 0
        batch_size = 10000
print('done')

done


In [10]:
## Сохранение результатов и расчет ошибки

In [11]:
y_test_values=y_test.values
predicted_results = model[0].predict_proba(f_matrix_test)
predicted_results = np.where(predicted_results[:,0]>predicted_results[:,1], 0.0,1.0)

sum_errors =sum(y_test_values - predicted_results)
print('count test values', len(y_test_values))
print('sum_errors', sum_errors)


count test values 4324
sum_errors 876.0


In [12]:
saved_result = pd.DataFrame({'text':test.values,
 'expected':  y_test_values,
 'predicted': predicted_results})

In [13]:
saved_result.to_csv('result.csv', encoding='utf-8', index=False)